In [1]:
import pandas as pd
import numpy as np
import pydub
from pydub import AudioSegment
from pydub.silence import detect_nonsilent
import pickle
from multiprocessing import Pool
import os
from datetime import datetime
from collections import namedtuple
from itertools import combinations
import random
from jiwer import wer

Range = namedtuple('Range',['start','end'])
os.chdir('/project2/mbspencer/nih/transcripts/')

In [2]:
# Set some defaults 
silence_buffer = 2500 # ms
#num_processors = 28
base_path = '/project2/graziul/LoP/corpus/transcriptions/WorkingData/'

In [3]:
today = datetime.today().strftime('%Y-%m-%d')
today = today.replace('-','_')
today

'2021_09_17'

#### For each file: Use intersection of transcriber utterance timings as threshold to create union of transcriber utterance timings, then use union to clip audio - add silence between, save file, receive test audio corpus with manual VAD feature extraction

In [4]:
df_transcripts = pd.read_csv('transcripts_raw2021_09_16.csv')
df_transcripts.head()

,feed,zone,year,month,day,time,sid,start,end,transcription,notes,file,transcriber,start_c,start_dt,end_c,end_dt,length,length_s,index
0,27730,Zone1,2018,8,12,932,RADIO,00.02.21.252,00.02.31.279,RADIOSHOP TESTING ONE TWO THREE FOUR FIVE FIVE...,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.02.21.252,1900-01-01 00:02:21.252000,00.02.31.279,1900-01-01 00:02:31.279000,0 days 00:00:10.027000,10.027,0
1,27730,Zone1,2018,8,12,932,RADIO,00.02.38.109,00.02.39.417,ONE TWO ONE TWO,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.02.38.109,1900-01-01 00:02:38.109000,00.02.39.417,1900-01-01 00:02:39.417000,0 days 00:00:01.308000,1.308,1
2,27730,Zone1,2018,8,12,932,UNIT,00.02.48.327,00.02.49.235,UNIT COMING IN,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.02.48.327,1900-01-01 00:02:48.327000,00.02.49.235,1900-01-01 00:02:49.235000,0 days 00:00:00.908000,0.908,2
3,27730,Zone1,2018,8,12,932,UNIT,00.02.55.330,00.02.57.437,ZONE ONE IS ON CITY [WIDE] FIVE,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.02.55.330,1900-01-01 00:02:55.330000,00.02.57.437,1900-01-01 00:02:57.437000,0 days 00:00:02.107000,2.107,3
4,27730,Zone1,2018,8,12,932,RADIO,00.03.04.003,00.03.09.017,OKAY THANKS UH THIS IS THE RADIO SHOP TESTING ...,NaN,201808120932-28710-27730ah.xlsx,huthealex,00.03.04.003,1900-01-01 00:03:04.003000,00.03.09.017,1900-01-01 00:03:09.017000,0 days 00:00:05.014000,5.014,4


In [5]:
df_transcripts[['filename','transcriber']] = df_transcripts['file'].str.split('(\d*-\d*-\d*)',expand=True)[[1,2]]
def remove_filetype(x):
    y = x.replace('.xlsx','')
    y = y.replace('.txt','')
    return y
df_transcripts['transcriber'] = df_transcripts['transcriber'].apply(lambda x: remove_filetype(x))
df_transcripts['transcriber'] = df_transcripts['transcriber'].str.lower().str.strip()
df_transcripts = df_transcripts[df_transcripts['transcriber']!='xxx']
#df_transcripts[['filename','transcriber']]
df_transcripts['transcriber'].value_counts()

sj     10408
wpd     6336
his     4900
ahs     4324
mgw     3422
tjs     3281
mh      3012
jar     2512
rah     2368
mcm     1760
kjc     1575
wam     1426
aek     1402
ah      1383
rc       698
oaa      519
_xz      385
kk       353
kw       239
Name: transcriber, dtype: int64

In [6]:
df_transcripts['start_dt'] = pd.to_datetime(df_transcripts['start_dt'])
df_transcripts['end_dt'] = pd.to_datetime(df_transcripts['end_dt'])

In [7]:
def remove_irrelevant(x):
    x = str(x)
    x = x.replace("\[Uncertain\]","")
    x = x.replace("INAUDIBLE","")    
    x = x.replace("<X>","")
    x = ' '.join(x.split())
    x = ''.join([i for i in x if i.isalnum() or i==' '])
    x = x.strip()
    x = x.upper()
    return x

In [8]:
df_transcripts['start_ts'] = (df_transcripts['start_dt']-np.datetime64('1900-01-01T00:00:00.000000000')).dt.total_seconds()
df_transcripts['end_ts'] = (df_transcripts['end_dt']-np.datetime64('1900-01-01T00:00:00.000000000')).dt.total_seconds()

In [9]:
# Some start/end are identical - remove these
print(len(df_transcripts[df_transcripts['start']==df_transcripts['end']]))
df_transcripts = df_transcripts[df_transcripts['start']!=df_transcripts['end']].copy()

51


In [10]:
df_transcripts['Range'] = df_transcripts[['start_ts','end_ts']].apply(tuple,axis=1)
df_transcripts.head()

,feed,zone,year,month,day,time,sid,start,end,transcription,...,start_dt,end_c,end_dt,length,length_s,index,filename,start_ts,end_ts,Range
0,27730,Zone1,2018,8,12,932,RADIO,00.02.21.252,00.02.31.279,RADIOSHOP TESTING ONE TWO THREE FOUR FIVE FIVE...,...,1900-01-01 00:02:21.252,00.02.31.279,1900-01-01 00:02:31.279,0 days 00:00:10.027000,10.027,0,201808120932-28710-27730,141.252,151.279,"(141.252, 151.279)"
1,27730,Zone1,2018,8,12,932,RADIO,00.02.38.109,00.02.39.417,ONE TWO ONE TWO,...,1900-01-01 00:02:38.109,00.02.39.417,1900-01-01 00:02:39.417,0 days 00:00:01.308000,1.308,1,201808120932-28710-27730,158.109,159.417,"(158.109, 159.417)"
2,27730,Zone1,2018,8,12,932,UNIT,00.02.48.327,00.02.49.235,UNIT COMING IN,...,1900-01-01 00:02:48.327,00.02.49.235,1900-01-01 00:02:49.235,0 days 00:00:00.908000,0.908,2,201808120932-28710-27730,168.327,169.235,"(168.327, 169.235)"
3,27730,Zone1,2018,8,12,932,UNIT,00.02.55.330,00.02.57.437,ZONE ONE IS ON CITY [WIDE] FIVE,...,1900-01-01 00:02:55.330,00.02.57.437,1900-01-01 00:02:57.437,0 days 00:00:02.107000,2.107,3,201808120932-28710-27730,175.330,177.437,"(175.33, 177.437)"
4,27730,Zone1,2018,8,12,932,RADIO,00.03.04.003,00.03.09.017,OKAY THANKS UH THIS IS THE RADIO SHOP TESTING ...,...,1900-01-01 00:03:04.003,00.03.09.017,1900-01-01 00:03:09.017,0 days 00:00:05.014000,5.014,4,201808120932-28710-27730,184.003,189.017,"(184.00300000000001, 189.01700000000002)"


In [11]:
def get_prop_overlap(se1, se2):
    s1, e1 = se1
    s2, e2 = se2
    overlap = max(0,min(e1,e2)-max(s1,s2))
    prop_overlap = overlap/max(e1-s1,e2-s2)
    return prop_overlap

In [12]:
def get_se_voice(filename, df_transcripts):
    df = df_transcripts[df_transcripts['filename']==f].copy()
    # Extract information so we can reconstruct dir_path
    zone = df['zone'].unique()[0]
    year = df['year'].astype(str).unique()[0]
    month = df['month'].astype(str).apply(lambda x: x.zfill(2)).unique()[0]
    day = df['day'].astype(str).apply(lambda x: x.zfill(2)).unique()[0]
    date = year+'_'+month+'_'+day
    # Create a list to hold the ranges we derive
    se_voice_list = []
    # Create a list of "exhausted" tuples (i.e. do not try to compare to others)
    exhausted = []
    # Get transcribers
    transcribers = df['transcriber'].unique().tolist()
    # Exclude aek as outlier transcriber
    transcribers = [i for i in transcribers if i not in ['aek','xxx']]
    # If only 1 transcription... do nothing
    # If 2+ transcriptions...
    if len(transcribers)>=2:
        # Pick 2 transcribers at random to avoid "issues"
        df_foruse = df[df['transcriber'].isin(random.sample(transcribers,2))].copy()
        # Get the ranges
        ranges = df_foruse['Range'].values
        # Check all permutations
        checked_se_tuples = []
        # Get target range
        for se1 in ranges:
            # Collect candidates
            candidate_se_list = []
            # Get start/end for target range
            s1, e1 = se1
            # Check overlap with remaining ranges
            for se2 in ranges:
                se_tuple = set((se1, se2))
                # Make sure it's not the target range and we have not checked for overlap of these ranges
                if se1!=se2: #) and len([i for i in checked_se_tuples if se_tuple in i])==0:
                    #checked_se_tuples.append(se_tuple)
                    prop_overlap = get_prop_overlap(se1, se2)
                   # If overlap more than 75%
                    if prop_overlap > 0.75 and [se2, se1] not in checked_se_tuples:
                        checked_se_tuples.append([se1, se2])
                        exhausted.append(se2)
                        exhausted.append(se1)
                        # Get start/end for other range
                        s2, e2 = se2
                        # Get the minimum start time of both periods 
                        start_voice = min(s1,s2)
                        # Get the maximum end time of both periods
                        end_voice = max(e1,e2)
                        # Get transcriptions associated with each range
                        t1 = df_foruse.loc[df['Range']==se1,'transcription'].apply(remove_irrelevant).values[0]
                        t2 = df_foruse.loc[df['Range']==se2,'transcription'].apply(remove_irrelevant).values[0]                   
                        # Add to candidate list
                        if t1 != '' and t2 != '':
                            # Get transcribers
                            p1 = df_foruse.loc[df['Range']==se1,'transcriber'].values[0]
                            p2 = df_foruse.loc[df['Range']==se2,'transcriber'].values[0]
                            candidate_se_list.append([zone, date, f+'.mp3', start_voice, end_voice, t1==t2, p1, t1, p2, t2])
            if len(candidate_se_list)>0:
                se_voice_list.append(candidate_se_list[0])
    return se_voice_list

In [13]:
%%time
voice_list = []
for f in df_transcripts['filename'].unique():
    voice_list += get_se_voice(f,df_transcripts)

CPU times: user 30.9 s, sys: 113 ms, total: 31 s
Wall time: 31 s


In [14]:
# Save voice activity file for use (includes transcriptions for determining difficulty)
df_va = pd.DataFrame(voice_list,columns=['zone','date','filename','start_voice','end_voice','transcripts_agree','transcriber1','transcription1','transcriber2','transcription2'])
df_va = df_va.drop_duplicates()
df_va.sort_values(['zone','date','start_voice'],inplace=True)
df_va.to_csv('df_va'+today+'.csv')
# Note: This represents number of utterances where both transcribers agree about presence of voice activity
#       and DOES NOT represent the total number of utterances identified by transribers
len(df_va)

6752

In [15]:
# Let's calculate %WER for each file as a more refined measure of difficulty (plan to use empiric dist anyway)
df_va_f = df_va.groupby(['zone','date','filename','transcriber1','transcriber2'])[['transcription1','transcription2']]
df_va_f_wer1 = df_va_f.apply(lambda x: wer(x['transcription1'].tolist(),x['transcription2'].tolist())).reset_index()
df_va_f_wer1 = df_va_f_wer1.rename(columns={0:'wer1vs2'})
df_va_f_wer1.head()

,zone,date,filename,transcriber1,transcriber2,wer1vs2
0,Zone1,2018_08_04,201808042331-339616-27730.mp3,mgw,kjc,0.305857
1,Zone1,2018_08_05,201808050001-100238-27730.mp3,kjc,wam,0.341338
2,Zone1,2018_08_05,201808050031-739899-27730.mp3,kjc,wam,0.289662
3,Zone1,2018_08_05,201808050100-298054-27730.mp3,mgw,wam,0.485417
4,Zone1,2018_08_05,201808050130-950381-27730.mp3,mgw,kjc,0.259594


In [16]:
df_va_f_wer2 = df_va_f.apply(lambda x: wer(x['transcription2'].tolist(),x['transcription1'].tolist())).reset_index()
df_va_f_wer2 = df_va_f_wer2.rename(columns={0:'wer2vs1'})
df_va_f_wer2.head()

,zone,date,filename,transcriber1,transcriber2,wer2vs1
0,Zone1,2018_08_04,201808042331-339616-27730.mp3,mgw,kjc,0.302575
1,Zone1,2018_08_05,201808050001-100238-27730.mp3,kjc,wam,0.340559
2,Zone1,2018_08_05,201808050031-739899-27730.mp3,kjc,wam,0.302998
3,Zone1,2018_08_05,201808050100-298054-27730.mp3,mgw,wam,0.476483
4,Zone1,2018_08_05,201808050130-950381-27730.mp3,mgw,kjc,0.246253


In [17]:
df_va_f_wer = df_va_f_wer1.merge(df_va_f_wer2)
df_va_f_wer['wer_avg'] = (df_va_f_wer['wer1vs2'] + df_va_f_wer['wer2vs1'])/2
df_va_f_wer['wer_avg'].describe()

count    134.000000
mean       0.410109
std        0.522976
min        0.000000
25%        0.253511
50%        0.304731
75%        0.386222
max        5.750000
Name: wer_avg, dtype: float64

In [18]:
# Sanity check, should all be equal
len(df_va_f_wer1), len(df_va_f_wer2), len(df_va_f_wer)

(134, 134, 134)

In [19]:
# Difficulty can be described by agreement, either binary (==) or word error rate (WER). 
# Using WER, we define "easier" audio to have WER in [25%,50%) and "harder" audio to have WER in [50%,75%)
cuts_dict = {i:df_va_f_wer['wer_avg'].describe().to_dict()[i] for i in ['25%','50%','75%']} 
wer_easier = [cuts_dict['25%'], cuts_dict['50%']]
wer_harder = [cuts_dict['50%'], cuts_dict['75%']]
wer_easier, wer_harder

([0.2535111524986559, 0.30473112946644465],
 [0.30473112946644465, 0.386221716612209])

In [20]:
df_file_difficulty = df_va_f_wer[['zone','date','filename','wer_avg']].copy()
df_file_difficulty.loc[(wer_easier[0]<=df_file_difficulty['wer_avg']) & (df_file_difficulty['wer_avg']<wer_easier[1]),'difficulty'] = 'easier'
df_file_difficulty.loc[(wer_harder[0]<=df_file_difficulty['wer_avg']) & (df_file_difficulty['wer_avg']<wer_harder[1]),'difficulty'] = 'harder'
df_file_difficulty['difficulty'].value_counts()

easier    33
harder    33
Name: difficulty, dtype: int64

In [21]:
easier_files = df_file_difficulty.loc[df_file_difficulty['difficulty']=='easier','filename'].tolist()
harder_files = df_file_difficulty.loc[df_file_difficulty['difficulty']=='harder','filename'].tolist()

### This code for creating test files using manually provided VAD data (test5, test6)

In [22]:
# Load old test data to ensure don't send repeats
df_test1 = pd.read_csv('ForGMR/test1.csv',index_col=0)
df_test2 = pd.read_csv('ForGMR/test2.csv',index_col=0)
df_test3 = pd.read_csv('ForGMR/test3.csv',index_col=0)

In [23]:
len(df_test1),len(df_test2), len(df_test3)

(126, 93, 85)

In [24]:
df_test123 = pd.concat([df_test1,df_test2,df_test3])
df_test123.head()

,zone,date,filename,start_voice,end_voice,transcripts_agree,transcription1,transcription2
2595,Zone1,2018_08_04,201808042331-339616-27730.mp3,70.952,73.086,True,SEVENTEEN TWENTY FOUR ROBERT,SEVENTEEN TWENTY FOUR ROBERT
2596,Zone1,2018_08_04,201808042331-339616-27730.mp3,73.712,75.073,True,TWENTY FOUR ROBERT,TWENTY FOUR ROBERT
2598,Zone1,2018_08_04,201808042331-339616-27730.mp3,81.790,82.598,True,TEN FOUR,TEN FOUR
2600,Zone1,2018_08_04,201808042331-339616-27730.mp3,93.638,95.388,True,SEVENTEEN THIRTEEN ROBERT,SEVENTEEN THIRTEEN ROBERT
2602,Zone1,2018_08_04,201808042331-339616-27730.mp3,118.949,122.450,True,SIXTEEN THIRTY FOUR ROBERT,SIXTEEN THIRTY FOUR ROBERT


In [25]:
len(df_test123)

304

In [26]:
# Create list of zone, date, filename, transcription1, and transcription2 to check for test data previously sent
test123_data = []
for _, row in df_test123.iterrows():
    test123_data.append([row['zone'],row['date'],row['filename'],row['transcription1']])
    test123_data.append([row['zone'],row['date'],row['filename'],row['transcription2']])
test123_data_df = pd.DataFrame(test123_data,columns=['zone','date','filename','transcription'])
df_exclude = test123_data_df.drop_duplicates()
df_exclude.head()

,zone,date,filename,transcription
0,Zone1,2018_08_04,201808042331-339616-27730.mp3,SEVENTEEN TWENTY FOUR ROBERT
2,Zone1,2018_08_04,201808042331-339616-27730.mp3,TWENTY FOUR ROBERT
4,Zone1,2018_08_04,201808042331-339616-27730.mp3,TEN FOUR
6,Zone1,2018_08_04,201808042331-339616-27730.mp3,SEVENTEEN THIRTEEN ROBERT
8,Zone1,2018_08_04,201808042331-339616-27730.mp3,SIXTEEN THIRTY FOUR ROBERT


In [27]:
df_va = pd.read_csv('df_va2021_09_17.csv')
df_va.drop(columns=df_va.columns.values[0],inplace=True)
df_va.head(10)

,zone,date,filename,start_voice,end_voice,transcripts_agree,transcriber1,transcription1,transcriber2,transcription2
0,Zone1,2018_08_04,201808042331-339616-27730.mp3,75.597,81.386,False,mgw,RESIDENTIAL ALARM FOURTY THIRTY ONE PATTERSON ...,kjc,RESIDENTIAL ALARM FORTY THIRTY ONE PATTERSON T...
1,Zone1,2018_08_04,201808042331-339616-27730.mp3,96.398,100.190,False,mgw,I NEED A RESULT FOR A BOOKING ON ARAGON ALBANY,kjc,CAN YOU PUT US DOWN FOR A UNCERTAIN ON UNCERTA...
2,Zone1,2018_08_04,201808042331-339616-27730.mp3,331.941,334.836,False,mgw,TRAFFIC STOP SIXTY ONE FOURTY FIVE NORTH WEST ...,kjc,TRAFFIC STOP AT TWO ONE FORTY FIVE STREET NAME...
3,Zone1,2018_08_04,201808042331-339616-27730.mp3,348.097,355.562,False,mgw,THEY SENT A FORCE TO INVOLVE CONCERN CITIZEN W...,kjc,THEY SAID A FOUR TWENTY FOUR OR FIVE CONCERNED...
4,Zone1,2018_08_04,201808042331-339616-27730.mp3,368.023,370.985,False,mgw,MO WITH DAL WITH SIXTEEN THIRTY THREE ROBERT S...,kjc,WELL LINK UP WITH SIXTEEN THIRTY THREE ROBERT ...
5,Zone1,2018_08_04,201808042331-339616-27730.mp3,371.523,373.206,True,mgw,SIXTEEN THIRTY THREE ROBERT,kjc,SIXTEEN THIRTY THREE ROBERT
6,Zone1,2018_08_04,201808042331-339616-27730.mp3,435.736,440.860,False,mgw,SIXTEEN TWENTY FIVE ON,kjc,ITS A FIFTY FOUR TWENTY FIVE UNCERTAIN
7,Zone1,2018_08_04,201808042331-339616-27730.mp3,446.515,450.285,False,mgw,TRAFFIC STOP FIFTY FOUR TWENTY FIVE ON KEDZIE,kjc,TRAFFIC STOP FIFTY FOUR TWENTY FIVE ON KEDSBY
8,Zone1,2018_08_04,201808042331-339616-27730.mp3,471.759,477.077,True,mgw,ONE SIX TWO FOUR ROBERT,kjc,ONE SIX TWO FOUR ROBERT
9,Zone1,2018_08_04,201808042331-339616-27730.mp3,477.750,486.501,False,mgw,TAKE A RIDE WITH FIRE FIFTY TWO O FIVE DIEGO B...,kjc,TAKE A RIDE WITH FIRE FIFTY TWO OH FIVE VEHICL...


##### Starting with df_va and df_exclude, remove transcriptions send in test1.csv, test2.csv, test3.csv to GMR

In [29]:
# Merge df_va with df_exclude, keeping only df_va rows so len(df_va)=len(no_test123step1)
no_test123step1 = df_va.merge(df_exclude,
                              left_on=['zone','date','filename','transcription1'],
                              right_on=['zone','date','filename','transcription'],
                              how='left',
                              indicator=True)
# Remove rows where df_exclude data were matched so len(no_test123step2)<len(no_test123step1)
no_test123step2 = no_test123step1[no_test123step1['_merge']=='left_only'].copy()
# Merge no_test123step2 with df_exclude, keeping only no_test123step2 rows so len(no_test123step2)=len(no_test123step3)
no_test123step3 = no_test123step2[[i for i in df_va.columns.values]].merge(df_exclude,
                              left_on=['zone','date','filename','transcription2'],
                              right_on=['zone','date','filename','transcription'],
                              how='left',
                              indicator=True)
# Remove rows where df_exclude data were matched so len(no_test123step4)<len(no_test123step3)
no_test123step4 = no_test123step3[no_test123step3['_merge']=='left_only'].copy()
df_va_notest123 = no_test123step4.drop(columns=['transcription','_merge'])
# Start with all df_va, note length of data to exclude, merge keeping only df_va rows 
len(df_va), len(df_exclude), len(no_test123step1), len(no_test123step2), len(no_test123step3), len(no_test123step4)

(6752, 383, 6752, 6608, 6608, 6600)

In [30]:
df_va_notest123.head()

,zone,date,filename,start_voice,end_voice,transcripts_agree,transcriber1,transcription1,transcriber2,transcription2
0,Zone1,2018_08_04,201808042331-339616-27730.mp3,75.597,81.386,False,mgw,RESIDENTIAL ALARM FOURTY THIRTY ONE PATTERSON ...,kjc,RESIDENTIAL ALARM FORTY THIRTY ONE PATTERSON T...
1,Zone1,2018_08_04,201808042331-339616-27730.mp3,96.398,100.190,False,mgw,I NEED A RESULT FOR A BOOKING ON ARAGON ALBANY,kjc,CAN YOU PUT US DOWN FOR A UNCERTAIN ON UNCERTA...
2,Zone1,2018_08_04,201808042331-339616-27730.mp3,331.941,334.836,False,mgw,TRAFFIC STOP SIXTY ONE FOURTY FIVE NORTH WEST ...,kjc,TRAFFIC STOP AT TWO ONE FORTY FIVE STREET NAME...
3,Zone1,2018_08_04,201808042331-339616-27730.mp3,348.097,355.562,False,mgw,THEY SENT A FORCE TO INVOLVE CONCERN CITIZEN W...,kjc,THEY SAID A FOUR TWENTY FOUR OR FIVE CONCERNED...
4,Zone1,2018_08_04,201808042331-339616-27730.mp3,368.023,370.985,False,mgw,MO WITH DAL WITH SIXTEEN THIRTY THREE ROBERT S...,kjc,WELL LINK UP WITH SIXTEEN THIRTY THREE ROBERT ...


##### Create test4.wav/test4.csv from files that are "easier"

In [31]:
selected_utterances4 = []
total_length = 0
# Create silence
silence = AudioSegment.silent(duration = silence_buffer, frame_rate = 22050)
combined = silence
files = [f for f in df_va_notest123['filename'].unique().tolist() if f in easier_files]
files.sort()
for file in files:
    if total_length < 10:
        # Get data associated with file
        df_temp = df_va_notest123[df_va_notest123['filename']==file].copy()
        # Get data needed to construct path to file
        zone = df_temp['zone'].unique()[0]
        date = df_temp['date'].unique()[0]
        # Load file
        try:
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        except:
            date_split = date.split('_')
            day = int(date_split[2])+1
            date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        # Get (start, end) tuples to extract voice activity
        se = df_temp[['start_voice','end_voice']].values
        # Build up audio segments with voice activity  
        for s,e in se:
            va_segment = pydub_audiosegment[max(0,s*1000-200):min(e*1000+200,len(pydub_audiosegment))]
            if len(detect_nonsilent(va_segment))>0:
                #print(len(detect_nonsilent(va_segment)),file)
                if total_length < 10: # and va_segment not in combined:
                    print("File #4 now "+str(round(total_length,1))+" in length",file)
                    combined += va_segment + silence
                    total_length = len(combined)/(1000*60)
                    selected_utterances4.append(df_temp[(df_temp['start_voice']==s) & (df_temp['end_voice']==e)].index)
        #del df_temp, se
wav_file_out = 'ForGMR/test4.wav'
combined = combined.set_frame_rate(22050)
combined.export(wav_file_out,format='wav')

File #4 now 0 in length 201808042331-339616-27730.mp3
File #4 now 0.2 in length 201808042331-339616-27730.mp3
File #4 now 0.3 in length 201808042331-339616-27730.mp3
File #4 now 0.4 in length 201808042331-339616-27730.mp3
File #4 now 0.6 in length 201808042331-339616-27730.mp3
File #4 now 0.7 in length 201808042331-339616-27730.mp3
File #4 now 0.8 in length 201808042331-339616-27730.mp3
File #4 now 1.0 in length 201808042331-339616-27730.mp3
File #4 now 1.1 in length 201808042331-339616-27730.mp3
File #4 now 1.3 in length 201808042331-339616-27730.mp3
File #4 now 1.4 in length 201808042331-339616-27730.mp3
File #4 now 1.5 in length 201808042331-339616-27730.mp3
File #4 now 1.6 in length 201808042331-339616-27730.mp3
File #4 now 1.8 in length 201808042331-339616-27730.mp3
File #4 now 1.8 in length 201808042331-339616-27730.mp3
File #4 now 1.9 in length 201808042331-339616-27730.mp3
File #4 now 2.0 in length 201808042331-339616-27730.mp3
File #4 now 2.1 in length 201808042331-339616-2773

<_io.BufferedRandom name='ForGMR/test4.wav'>

In [37]:
df_va_notest123[df_va_notest123.index.isin([i[0] for i in selected_utterances4])].to_csv('ForGMR/test4.csv')

##### Create test5.wav/test5.csv from files that are "harder"

In [38]:
selected_utterances5 = []
total_length = 0
# Create silence
silence = AudioSegment.silent(duration = silence_buffer, frame_rate = 22050)
combined = silence
files = [f for f in df_va_notest123['filename'].unique().tolist() if f in harder_files]
files.sort()
for file in files:
    if total_length < 10:
        # Get data associated with file
        df_temp = df_va_notest123[df_va_notest123['filename']==file].copy()
        # Get data needed to construct path to file
        zone = df_temp['zone'].unique()[0]
        date = df_temp['date'].unique()[0]
        # Load file
        try:
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        except:
            date_split = date.split('_')
            day = int(date_split[2])+1
            date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        # Get (start, end) tuples to extract voice activity
        se = df_temp[['start_voice','end_voice']].values
        # Build up audio segments with voice activity  
        for s,e in se:
            va_segment = pydub_audiosegment[max(0,s*1000-200):min(e*1000+200,len(pydub_audiosegment))]
            if len(detect_nonsilent(va_segment))>0:
                #print(len(detect_nonsilent(va_segment)),file)
                if total_length < 10: # and va_segment not in combined:
                    print("File #5 now "+str(round(total_length,1))+" in length",file)
                    combined += va_segment + silence
                    total_length = len(combined)/(1000*60)
                    selected_utterances5.append(df_temp[(df_temp['start_voice']==s) & (df_temp['end_voice']==e)].index)
        #del df_temp, se
wav_file_out = 'ForGMR/test5.wav'
combined = combined.set_frame_rate(22050)
combined.export(wav_file_out,format='wav')

File #5 now 0 in length 201808050001-100238-27730.mp3
File #5 now 0.2 in length 201808050001-100238-27730.mp3
File #5 now 0.2 in length 201808050001-100238-27730.mp3
File #5 now 0.4 in length 201808050001-100238-27730.mp3
File #5 now 0.5 in length 201808050001-100238-27730.mp3
File #5 now 0.6 in length 201808050001-100238-27730.mp3
File #5 now 0.7 in length 201808050001-100238-27730.mp3
File #5 now 0.8 in length 201808050001-100238-27730.mp3
File #5 now 0.9 in length 201808050001-100238-27730.mp3
File #5 now 1.0 in length 201808050001-100238-27730.mp3
File #5 now 1.1 in length 201808050001-100238-27730.mp3
File #5 now 1.2 in length 201808050001-100238-27730.mp3
File #5 now 1.3 in length 201808050001-100238-27730.mp3
File #5 now 1.4 in length 201808050001-100238-27730.mp3
File #5 now 1.5 in length 201808050001-100238-27730.mp3
File #5 now 1.6 in length 201808050001-100238-27730.mp3
File #5 now 1.7 in length 201808050001-100238-27730.mp3
File #5 now 1.7 in length 201808050001-100238-2773

<_io.BufferedRandom name='ForGMR/test5.wav'>

In [39]:
df_va_notest123[df_va_notest123.index.isin([i[0] for i in selected_utterances5])].to_csv('ForGMR/test5.csv')

In [40]:
df_va_notest123.to_csv('ForGMR/df_va_ForTest45.csv')

### This code for file-by-file removal of silence using manually provided VAD data

In [ ]:
def get_all_voice_activity(df_va):
    p = Pool(processes = num_processors)
    output = p.starmap(extract_voice_activity,[(file,df_va) for file in df_va['filename'].unique()])
    p.close()

In [ ]:
def extract_voice_activity(file, df_va, padding=200):
    # Get data associated with file
    df_temp = df_va[df_va['filename']==file].copy()
    # Get data needed to construct path to file
    zone = df_temp['zone'].unique()[0]
    date = df_temp['date'].unique()[0]
    # Load file
    try:
        mp3_file_in = base_path+zone+'/'+date+'/'+file
        pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
    except:
        date_split = date.split('_')
        day = int(date_split[2])+1
        date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
        mp3_file_in = base_path+zone+'/'+date+'/'+file
        pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
    # Get frame_rate
    fr = pydub_audiosegment.frame_rate
    # Create silence
    silence = AudioSegment.silent(duration = silence_buffer, frame_rate = fr)
    # Get (start, end) tuples to extract voice activity
    se = df_temp[['start_voice','end_voice']].sort_values('start_voice').values
    # Build up list of audio segments with voice activity  
    va_slices = []
    for s,e in se: 
        va_slices.append(pydub_audiosegment[max(0,s*1000-padding):min(e*1000+padding,len(pydub_audiosegment))])
    combined = silence
    for va_slice in va_slices:
        combined += va_slice + silence
    wav_file_out = base_path+zone+'/'+date+'/'+file.split('.')[0]+'va.wav'
    combined = combined.set_frame_rate(fr)
    combined.export(wav_file_out,format='wav')

In [ ]:
%%time
get_all_voice_activity(df_va)

### This code for creating test files using manually provided VAD data (test1, test2, test3)

##### NOTE: Because of stochastic selection of pairs of transcribers, files cannot be reproduced. Code maintained here for future reference.

In [ ]:
# Now use these timings to compile sample data for use by GMR and Deepgram
#  1. 3 x 10 min files for GMR
#  2. "a couple hours" for Deepgram
#  3. Preserve temporal order

In [ ]:
# File #1: Both transcribers agree ("clean" = easy)
df_va1 = df_va[df_va['transcripts_agree']].copy()
df_va1 = df_va1.drop_duplicates()
df_va1 = df_va1[~df_va1['transcription1'].isna()]
len(df_va1)

In [ ]:
df_va1.sort_values(['date','start_voice'],inplace=True)
# Exhausted case indices
exhausted_cases = []
total_length = 0
# Create silence
silence = AudioSegment.silent(duration = silence_buffer, frame_rate = 22050)
combined = silence
files = df_va1['filename'].unique().tolist()
files.sort()
for file in files:
    if total_length < 10:
        # Get data associated with file
        df_temp = df_va1[df_va1['filename']==file].copy()
        # Get data needed to construct path to file
        zone = df_temp['zone'].unique()[0]
        date = df_temp['date'].unique()[0]
        # Load file
        try:
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        except:
            date_split = date.split('_')
            day = int(date_split[2])+1
            date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        # Get (start, end) tuples to extract voice activity
        se = df_temp[['start_voice','end_voice']].values
        # Build up audio segments with voice activity  
        for s,e in se:
            va_segment = pydub_audiosegment[max(0,s*1000-200):min(e*1000+200,len(pydub_audiosegment))]
            if len(detect_nonsilent(va_segment))>0:
                #print(len(detect_nonsilent(va_segment)),file)
                if total_length < 10: # and va_segment not in combined:
                    print("File #1 now "+str(round(total_length,1))+" in length",file)
                    combined += va_segment + silence
                    total_length = len(combined)/(1000*60)
                    exhausted_cases.append(df_temp[(df_temp['start_voice']==s) & (df_temp['end_voice']==e)].index)
        #del df_temp, se
wav_file_out = base_path+'test1.wav'
combined = combined.set_frame_rate(22050)
combined.export(wav_file_out,format='wav')

In [ ]:
exhausted_from_test1 = pd.Index([i for s in exhausted_cases for i in s])
len(exhausted_cases), len(df_va), len(df_va[~df_va.index.isin(exhausted_from_test1)])

In [ ]:
df_va[df_va.index.isin(exhausted_from_test1)].to_csv('test1.csv')

In [ ]:
# File #2: Mix of agree/disagree ("not clean" = medium)
df_va2 = df_va[(~df_va.index.isin(exhausted_from_test1)) & (df_va.index>max(exhausted_from_test1))].copy()
print("Proportion of samples where transcripts disagree:", str(len(df_va2[~df_va2['transcripts_agree']])/len(df_va2)))
print("Length of df_va2: ",str(len(df_va2)))

In [ ]:
df_va2.sort_values(['date','start_voice'],inplace=True)
df_va2.head(20)

In [ ]:
df_va2.sort_values(['date','start_voice'],inplace=True)
# Exhausted case indices
exhausted_cases = []
total_length = 0
# Create silence
silence = AudioSegment.silent(duration = silence_buffer, frame_rate = 22050)
combined = silence
files = df_va2['filename'].unique().tolist()
files.sort()
for file in files:
    if total_length < 10:
        # Get data associated with file
        df_temp = df_va2[df_va2['filename']==file].copy()
        # Get data needed to construct path to file
        zone = df_temp['zone'].unique()[0]
        date = df_temp['date'].unique()[0]
        # Load file
        try:
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        except:
            date_split = date.split('_')
            day = int(date_split[2])+1
            date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        # Get (start, end) tuples to extract voice activity
        se = df_temp[['start_voice','end_voice']].values
        # Build up audio segments with voice activity  
        for s,e in se:
            va_segment = pydub_audiosegment[max(0,s*1000-200):min(e*1000+200,len(pydub_audiosegment))]
            if len(detect_nonsilent(va_segment))>0:
                #print(len(detect_nonsilent(va_segment)),file)
                if total_length < 10: # and va_segment not in combined:
                    print("File #2 now "+str(round(total_length,1))+" in length",file)
                    combined += va_segment + silence
                    total_length = len(combined)/(1000*60)
                    exhausted_cases.append(df_temp[(df_temp['start_voice']==s) & (df_temp['end_voice']==e)].index)
        #del df_temp, se
wav_file_out = base_path+'test2.wav'
combined = combined.set_frame_rate(22050)
combined.export(wav_file_out,format='wav')

In [ ]:
exhausted_from_test2 = pd.Index([i for s in exhausted_cases for i in s])
len(exhausted_from_test1), len(exhausted_from_test2), len(df_va), len(df_va2), len(df_va[~df_va.index.isin(exhausted_from_test1.union(exhausted_from_test2))])

In [ ]:
df_va[df_va.index.isin(exhausted_from_test2)].to_csv('test2.csv')

In [ ]:
# File #3: Transcribers disagree ("not clean" = hard)'
df_va3 = df_va[(~df_va['transcripts_agree']) & (df_va.index>max(exhausted_from_test2)-500)].copy()
print("Proportion of samples where transcripts disagree:", str(len(df_va3[~df_va3['transcripts_agree']])/len(df_va3)))
print("Length of df_va3: ",str(len(df_va3)))

In [ ]:
df_va3.sort_values(['date','start_voice'],inplace=True)
df_va3.head(20)

In [ ]:
df_va3.sort_values(['date','start_voice'],inplace=True)
# Exhausted case indices
exhausted_cases = []
total_length = 0
# Create silence
silence = AudioSegment.silent(duration = silence_buffer, frame_rate = 22050)
combined = silence
files = df_va3['filename'].unique().tolist()
files.sort()
for file in files:
    if total_length < 10:
        # Get data associated with file
        df_temp = df_va3[df_va3['filename']==file].copy()
        # Get data needed to construct path to file
        zone = df_temp['zone'].unique()[0]
        date = df_temp['date'].unique()[0]
        # Load file
        try:
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        except:
            date_split = date.split('_')
            day = int(date_split[2])+1
            date = date_split[0]+'_'+date_split[1]+'_'+str(day).zfill(2)
            mp3_file_in = base_path+zone+'/'+date+'/'+file
            pydub_audiosegment = AudioSegment.from_mp3(mp3_file_in)
        # Get (start, end) tuples to extract voice activity
        se = df_temp[['start_voice','end_voice']].values
        # Build up audio segments with voice activity  
        for s,e in se:
            va_segment = pydub_audiosegment[max(0,s*1000-200):min(e*1000+200,len(pydub_audiosegment))]
            if len(detect_nonsilent(va_segment))>0:
                #print(len(detect_nonsilent(va_segment)),file)
                if total_length < 10: # and va_segment not in combined:
                    print("File #3 now "+str(round(total_length,1))+" in length",file)
                    combined += va_segment + silence
                    total_length = len(combined)/(1000*60)
                    exhausted_cases.append(df_temp[(df_temp['start_voice']==s) & (df_temp['end_voice']==e)].index)
        #del df_temp, se
wav_file_out = base_path+'test3.wav'
combined = combined.set_frame_rate(22050)
combined.export(wav_file_out,format='wav')

In [ ]:
exhausted_from_test3 = pd.Index([i for s in exhausted_cases for i in s])

In [ ]:
df_va[df_va.index.isin(exhausted_from_test3)].to_csv('test3.csv')